<a href="https://colab.research.google.com/github/Fridman77/LFPC-Labs/blob/main/Lab4LFPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Varianta 17



```
G=(VN, VT, P, S) 
VN={S, A, B, C, D, E} VT={a, b}
P={
1. S->aA
2. S->AC
3. A->a
4. A->ASC
5. A->BC
6. A->aD
7. B->b
8. B->bA
9. C->ε
10. C->BA
11. E->aB
12. D->abC
}
```




In [2]:
from collections import defaultdict
from itertools import combinations

In [3]:
Vn = ['S','A','B','C','D','E']
Vt = ['a','b']
  
productions = ['S->aA','S->AC','A->a','A->ASC','A->BC','A->aD','B->b','B->bA','C->ε','C->BA','E->aB','D->abC']

In [4]:
def parseGrammar():
  grammar = {}
  
  for transtions in productions:
    rules = transtions.split("->")
    rule = list(rules[1])
    if not rules[0] in grammar:
      grammar[rules[0]] = []
    grammar[rules[0]].append(rule)
    
  return defaultdict(lambda: "default",grammar=grammar, Vn=Vn, Vt=Vt)

parsedGrammar = parseGrammar()

In [5]:
def checkEpsilon(grammar):
  epsilon_rules = []

  for state in grammar:
    for transitions in grammar[state]:
      if '$' in transitions and len(transitions) == 1:
        epsilon_rules.append(state)
  return epsilon_rules

In [6]:
# Check the frequence of a non-terminal in a production.
def checkFrequency(transitions, non_term, symbol_freq):
  freq = 0;
  for index in range(len(transitions)):
    if transitions[index] == non_term:
      freq += 1
      if freq == symbol_freq:
        return index

In [9]:
# Create all possible subsets of a production.
def subsets(transitions, remove_transition, has_epsilon):
  # Init array for possible combinations
  possible_combinations = []
  for x in range(1, has_epsilon + 1):
    possible_combinations += list(combinations(range(1, has_epsilon + 1), x))
    
    new_transitions = []
  for combination in possible_combinations:
    # Copy the transition to compute.
    comb = transitions.copy()
    for number in combination:
      # Remove the empty state to create a new combination.
      comb.pop(checkFrequency(transitions, remove_transition, number) - len(transitions) + len(comb))
    # If the combination doesn't exists - add it to the list.
    if comb not in new_transitions:
      new_transitions.append(comb)

  return new_transitions

In [10]:
def removeEpsilon(grammar):
    
  while(len(checkEpsilon(grammar))):
    epsilon_rules = checkEpsilon(grammar)
    
    for derives_eps in epsilon_rules:
      for state in grammar:
        for transitions in grammar[state]:
          # Check if the empty state is in any right production.
          if derives_eps in transitions:
            # Count how many empty states are in the production.
            has_epsilon = transitions.count(derives_eps)

            # If all the rules derives to the empty state - make the current state empty.
            if has_epsilon == len(transitions):
              grammar[state].append(['$'])
              continue

            # Compute the possible combinations.
            possibleCombinations = subsets(transitions, derives_eps, has_epsilon)
            # Update the grammar with the new transitions.
            for comb in possibleCombinations:
              if comb not in grammar[state]:
                grammar[state].append(comb)

      # Remove epsilon transitions.
      grammar[derives_eps].remove(['$'])
  return grammar

In [11]:
def removeUnit(grammar, Vn):
  for state in grammar:
    for transitions in grammar[state]:
      # Check if the states derives in a single Non-Terminal Symbol.
      if len(transitions) == 1 and transitions[0] in Vn:
        # If true - remove it.
        grammar[state].remove(transitions)
        for transition_rules in grammar[transitions[0]]:
          if transition_rules not in grammar[state]:
            # Update the current state' transition to the adj state transition.
            grammar[state].append(transition_rules)
  return grammar

In [12]:
def removeInaccessible(grammar, Vn):
  non_terminals = Vn.copy();
  non_terminals.remove("S")

  for non_terminal in non_terminals:
    inaccessible = True

    for state in grammar:
      for transitions in grammar[state]:
        if non_terminal in transitions and state != non_terminal:
            inaccessible = False
            break
    
    if inaccessible:
      grammar.pop(non_terminal)
  return grammar


In [13]:
print('\nInput', parsedGrammar['grammar'])


Input {'S': [['a', 'A'], ['A', 'C']], 'A': [['a'], ['A', 'S', 'C'], ['B', 'C'], ['a', 'D']], 'B': [['b'], ['b', 'A']], 'C': [['ε'], ['B', 'A']], 'E': [['a', 'B']], 'D': [['a', 'b', 'C']]}


In [14]:
print('\nStep 1: Eliminate epsilon productions')
eliminate_eps_prod = removeEpsilon(parsedGrammar['grammar'])
print(eliminate_eps_prod)


Step 1: Eliminate epsilon productions
{'S': [['a', 'A'], ['A', 'C']], 'A': [['a'], ['A', 'S', 'C'], ['B', 'C'], ['a', 'D']], 'B': [['b'], ['b', 'A']], 'C': [['ε'], ['B', 'A']], 'E': [['a', 'B']], 'D': [['a', 'b', 'C']]}


In [15]:
print('\nStep 2: Eliminate unit productions')
eliminate_unit_prod = removeUnit(eliminate_eps_prod, parsedGrammar['Vn'])
print(eliminate_unit_prod)


Step 2: Eliminate unit productions
{'S': [['a', 'A'], ['A', 'C']], 'A': [['a'], ['A', 'S', 'C'], ['B', 'C'], ['a', 'D']], 'B': [['b'], ['b', 'A']], 'C': [['ε'], ['B', 'A']], 'E': [['a', 'B']], 'D': [['a', 'b', 'C']]}


In [16]:
print('\nStep 3: Eliminate inaccesible productions')
eliminate_inaccessible_prod = removeInaccessible(eliminate_unit_prod, parsedGrammar['Vn'])
print(eliminate_inaccessible_prod)


Step 3: Eliminate inaccesible productions
{'S': [['a', 'A'], ['A', 'C']], 'A': [['a'], ['A', 'S', 'C'], ['B', 'C'], ['a', 'D']], 'B': [['b'], ['b', 'A']], 'C': [['ε'], ['B', 'A']], 'D': [['a', 'b', 'C']]}
